<a href="https://colab.research.google.com/github/allnes/pcam_train/blob/master/pcam_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
cd  '/content/drive/My Drive/DL_PCAM'

/content/drive/My Drive/DL_PCAM


In [69]:
%ls

camelyonpatch_level_2_split_test_meta.csv
camelyonpatch_level_2_split_test_x.h5
camelyonpatch_level_2_split_test_x.h5.gz
camelyonpatch_level_2_split_test_y.h5
camelyonpatch_level_2_split_test_y.h5.gz
camelyonpatch_level_2_split_train_meta.csv
camelyonpatch_level_2_split_train_x.h5
camelyonpatch_level_2_split_train_x.h5.gz
camelyonpatch_level_2_split_train_y.h5
camelyonpatch_level_2_split_train_y.h5.gz
camelyonpatch_level_2_split_valid_meta.csv
camelyonpatch_level_2_split_valid_x.h5
camelyonpatch_level_2_split_valid_x.h5.gz
camelyonpatch_level_2_split_valid_y.h5
camelyonpatch_level_2_split_valid_y.h5.gz
__pycache__/


In [0]:
import gzip
import shutil
def uzip_file(file_name, origin_name):
  with gzip.open(file_name, 'rb') as f_in:
      with open(origin_name, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)


In [0]:
train_x_name = 'camelyonpatch_level_2_split_train_x.h5'
train_y_name = 'camelyonpatch_level_2_split_train_y.h5'

test_x_name = 'camelyonpatch_level_2_split_test_x.h5'
test_y_name = 'camelyonpatch_level_2_split_test_y.h5'

valid_x_name = 'camelyonpatch_level_2_split_valid_x.h5'
valid_y_name = 'camelyonpatch_level_2_split_valid_y.h5'

meta_train_name = 'camelyonpatch_level_2_split_train_meta.csv'
meta_valid_name = 'camelyonpatch_level_2_split_valid_meta.csv'
meta_test_name  = 'camelyonpatch_level_2_split_test_meta.csv'

# uzip_file(train_x_name + '.gz', train_x_name)
# uzip_file(train_y_name + '.gz', train_y_name)

# uzip_file(test_x_name + '.gz', test_x_name)
# uzip_file(test_y_name + '.gz', test_y_name)

# uzip_file(valid_x_name + '.gz', valid_x_name)
# uzip_file(valid_y_name + '.gz', valid_y_name)


In [0]:
from keras.utils import HDF5Matrix
import pandas as pd

x_train = HDF5Matrix(train_x_name, 'x', start=10000, end=20000)
y_train = HDF5Matrix(train_y_name, 'y', start=10000, end=20000)

x_valid = HDF5Matrix(valid_x_name, 'x', start=10000, end=20000)
y_valid = HDF5Matrix(valid_y_name, 'y', start=10000, end=20000)

x_test = HDF5Matrix(test_x_name, 'x')
y_test = HDF5Matrix(test_y_name, 'y')

meta_train = pd.read_csv(meta_train_name)
meta_valid = pd.read_csv(meta_valid_name)
meta_test = pd.read_csv(meta_test_name)

In [73]:
print(len(x_train))
print(type(x_train))
print(type(x_train[0]))
print(x_train[0].shape)


10000
<class 'keras.utils.io_utils.HDF5Matrix'>
<class 'numpy.ndarray'>
(96, 96, 3)


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape, Lambda, Input
from keras.regularizers import l2
import tensorflow as tf

In [0]:
full_size = 32 * 32 * 3
original_shape = (96, 96, 3)
new_shape = (32, 32)

In [76]:
model = Sequential([
    # Input(original_shape),
    Lambda(lambda image: tf.image.resize(image, new_shape)),
    Reshape((1, 1, full_size,), input_shape=x_train[0].shape),
    Dense(16384), Activation('relu'),
    # Dense(5000),  Activation('relu'),
    Dense(1),     Activation('softmax'),
])
model.build((len(x_train), 96, 96, 3))
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_11 (Lambda)           (10000, 32, 32, 3)        0         
_________________________________________________________________
reshape_11 (Reshape)         (10000, 1, 1, 3072)       0         
_________________________________________________________________
dense_22 (Dense)             (10000, 1, 1, 16384)      50348032  
_________________________________________________________________
activation_22 (Activation)   (10000, 1, 1, 16384)      0         
_________________________________________________________________
dense_23 (Dense)             (10000, 1, 1, 1)          16385     
_________________________________________________________________
activation_23 (Activation)   (10000, 1, 1, 1)          0         
Total params: 50,364,417
Trainable params: 50,364,417
Non-trainable params: 0
_________________________________________

In [0]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])

In [0]:
model.fit(x_train, y_train, epochs=10, batch_size=10000, shuffle='batch', validation_data=(x_valid, y_valid))

Train on 10000 samples, validate on 10000 samples
Epoch 1/10
10000/10000 [==============================] - 70s 7ms/step - loss: 7.9010 - acc: 0.5044 - val_loss: 7.9537 - val_acc: 0.5011
Epoch 2/10


In [0]:
_, accuracy = model.evaluate(x_train, y_train)
print('Accuracy: %.2f' % (accuracy * 100))